<a href="https://colab.research.google.com/github/ExploitIdF/analyse_preventifs_23_25/blob/main/tunMarCoefFreqEq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

On veut créer un document qui extrait les formules des références prix du tableau de Fanny, et où l'on peut mettre une colonne des coéfficients pour chaque référence prix extraite, et aussi le nombre équipé et la fréquence totale. Ainsi, on pourra comparer les montants de 2023 et 2025 aux normatives de 2025 au niveau des prix.

In [1]:
# Importer les bibliothèques nécessaires
import re
import pandas as pd
from openpyxl import load_workbook
import requests
from io import BytesIO
import numpy as np

# Affichage complet dans pandas
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# Télécharger le fichier Excel depuis l'URL
url = 'https://storage.googleapis.com/sucombe-dirif/reference/Tunnels_Cout%20pr%C3%A9ventif_04.04.2024.xlsx'
response = requests.get(url)
file_stream = BytesIO(response.content)

# Charger le fichier Excel avec openpyxl
wb = load_workbook(filename=file_stream, data_only=False)
ref_sheet = 'Préventifs_tunnels'
ref_ws = wb[ref_sheet]

In [18]:
prevTun = pd.read_excel(url,sheet_name=ref_sheet)

In [19]:
prevTun[:1]

,Numéro\nmarché,Marché,Type Opération,Référence\nprix,Détail Opération,Secteur\ngéographique,Prix\nactuel,Prix unitaire\nHT PF,Fin validité\ndu prix PF,Prix unitaire\nHT R1,Fin validité\ndu prix R1,Prix unitaire\nHT R2,Fin validité\ndu prix R2,Prix unitaire\nHT R3,Fin validité\ndu prix R3,Fréq\nDE,Unnamed: 16
0,2033090,Bâtiment,Préventif IS + niches,MPE001,EST - Inspection de maintenance préventive des issues et niches dans le tunnel de Boissy (sens W),Boissy,900.0,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1900-01-01,1.0,NaN


In [3]:
# Liste des feuilles Excel des tunnels
tunnel_sheets = [
    'Boissy', 'Champigny', 'Guy Môquet', 'Moulin', 'Nogent',
    'Ambroise Paré', 'Belle-Rive', 'Chennevières', 'Fontenay', 'La Défense',
    'Nanterre Centre', 'Nanterre échangeur', 'Neuilly', 'Saint-Cloud', 'Sévines',
    'Bobigny', 'La Courneuve', 'Landy', 'Lumen-Norton', 'Taverny',
    'Antony', 'Fresnes', 'Bicêtre', 'Italie', 'Orly',
]

# Types de marchés recherchés
marches = ["Bâtiment", "Propreté", "ContReg", "Eclairage", "AEV", "Automate", "PAU/TSE", "MEC", "Onduleur", "Détection", "Ventilation", "Vidéo", "Pompage"]

In [4]:
# Obtenir la valeur de référence prix depuis l'onglet global 'Préventifs_tunnels'
def get_ref_prix(row_num):
    try:
        val = ref_ws[f"D{row_num}"].value
        return val if val else f"MISSING D{row_num}"
    except:
        return f"INVALID D{row_num}"


In [5]:
# Obtenir la valeur de référence prix depuis la feuille de référence
def get_ref_prix(row_num):
    try:
        val = ref_ws[f"D{row_num}"].value
        return val if val else f"MISSING D{row_num}"
    except:
        return f"INVALID D{row_num}"


Il faut lire les formules de la colonne G dans l'onglet de chaque tunnel maintenant.

In [6]:
# Extraire les détails d'une formule Excel
def extract_formula_details(formula):
    if re.match(r"^=\s*'?.+?!G\d+$", formula):
        return 'direct', [(formula.split('!G')[-1])], 1.0
    elif re.match(r"^=\(\s*'?.+?!G\d+\s*\+\s*'?.+?!G\d+\s*\)$", formula):
        return 'sum_2', re.findall(r"G(\d+)", formula), 1.0
    elif re.match(r"(?i)^=\s*SUM\(\s*'?.+?!G(\d+):G(\d+)\s*\)$", formula):
        match = re.search(r"G(\d+):G(\d+)", formula)
        if match:
            start, end = int(match.group(1)), int(match.group(2))
            return f"sum_{abs(end - start) + 1}", [str(i) for i in range(start, end + 1)], 1.0
    elif re.match(r"^=\(\s*'?.+?!G\d+\s*\+\s*'?.+?!G\d+\s*\)/\d+(\.\d+)?$", formula):
        return 'div_2', re.findall(r"G(\d+)", formula), 0.5
    elif re.match(r"^=\s*'?.+?!G\d+/\d+(\.\d+)?$", formula):
        return 'div_1', [re.findall(r"G(\d+)", formula)[0]], 0.5
    else:
        return 'unknown', [], 0


Les formules sont soit des sommes, soit des divisions, soit des références à des autres cellules dans une autre feuille; donc, il faut qu'on prend tout ça en compte quand on lit les formules.

In [7]:
# Résoudre une valeur de cellule Excel avec formules, récursivement
def resolve_cell_value(wb, ws, val, max_depth=5):
    sheet_name_map = {'Equipementspartunnel': 'Equipements par tunnel'}
    if max_depth <= 0:
        return f"Max recursion depth reached"
    if not isinstance(val, str) or not val.startswith('='):
        return val

    val_clean = val.lstrip('=').replace(' ', '')

    # Gestion des formules SUM
    match_sum = re.match(r"(?i)^SUM\(([A-Z]+\d+):([A-Z]+\d+)\)$", val_clean)
    if match_sum:
        col = re.match(r"([A-Z]+)", match_sum.group(1)).group(1)
        start_row = int(re.match(r"[A-Z]+(\d+)", match_sum.group(1)).group(1))
        end_row = int(re.match(r"[A-Z]+(\d+)", match_sum.group(2)).group(1))
        total = 0
        for r in range(start_row, end_row + 1):
            v = resolve_cell_value(wb, ws, ws[f"{col}{r}"].value, max_depth - 1)
            try:
                total += float(v)
            except:
                pass
        return total

    # Gestion des opérations entre cellules (mêmes ou autres feuilles)
    op_match = re.match(r"(?:'([^']+)')?!?([A-Z]+\d+)([\+\-\/])(?:'([^']+)')?!?([A-Z]+\d+)", val_clean)
    if op_match:
        sheet1, cell1, op, sheet2, cell2 = op_match.groups()
        sheet1 = sheet_name_map.get(sheet1, sheet1) if sheet1 else ws.title
        sheet2 = sheet_name_map.get(sheet2, sheet2) if sheet2 else ws.title
        val1 = resolve_cell_value(wb, wb[sheet1], wb[sheet1][cell1].value, max_depth - 1) if sheet1 in wb.sheetnames else 0
        val2 = resolve_cell_value(wb, wb[sheet2], wb[sheet2][cell2].value, max_depth - 1) if sheet2 in wb.sheetnames else 0
        try:
            val1, val2 = float(val1), float(val2)
            return val1 + val2 if op == '+' else val1 - val2 if op == '-' else val1 / val2 if val2 != 0 else 0
        except:
            return 0

    # Référence simple vers une autre cellule d'une autre feuille
    match_ref = re.match(r"'?([^']+)'?!([A-Z]+\d+)", val_clean)
    if match_ref:
        sheet_name, cell_ref = match_ref.groups()
        sheet_name = sheet_name_map.get(sheet_name, sheet_name)
        if sheet_name in wb.sheetnames:
            cell_val = wb[sheet_name][cell_ref].value
            return resolve_cell_value(wb, wb[sheet_name], cell_val, max_depth - 1)
        else:
            return f"MISSING SHEET {sheet_name}"

    # Références locales avec opérateurs
    match_local = re.match(r"([A-Z]+\d+)([\+\-\/])([A-Z]+\d+)", val_clean)
    if match_local:
        c1, op, c2 = match_local.groups()
        v1 = resolve_cell_value(wb, ws, ws[c1].value, max_depth - 1)
        v2 = resolve_cell_value(wb, ws, ws[c2].value, max_depth - 1)
        try:
            v1, v2 = float(v1), float(v2)
            return v1 + v2 if op == '+' else v1 - v2 if op == '-' else v1 / v2 if v2 != 0 else 0
        except:
            return 0

    return val


In [8]:
# Extraire les formules et valeurs depuis toutes les feuilles
all_rows = []

for tunnel in tunnel_sheets:
    ws = wb[tunnel]
    for row in ws.iter_rows(min_row=3, min_col=1, max_col=7):
        marche_cell, g_cell = row[0], row[6]
        if marche_cell.value not in marches or g_cell.data_type != 'f':
            continue

        marche = marche_cell.value
        formula = g_cell.value
        formula_type, row_refs, coeff = extract_formula_details(formula)
        freq_totale_val = resolve_cell_value(wb, ws, row[4].value)
        nbr_equipe_val = resolve_cell_value(wb, ws, row[5].value)
        try:
            nbr_equipe_val = float(nbr_equipe_val)
        except:
            nbr_equipe_val = 0

        if not row_refs:
            all_rows.append({
                'Tunnel': tunnel,
                'Marché': marche,
                'Référence prix': '',
                'Coefficient': coeff,
                'freq_totale': freq_totale_val,
                'nbr_equipe': nbr_equipe_val,
                'Source Cell': g_cell.coordinate,
                'Formula Type': formula_type
            })
            continue

        for i, ref_row in enumerate(row_refs):
            ref_prix = get_ref_prix(int(ref_row))
            all_rows.append({
                'Tunnel': tunnel,
                'Marché': marche,
                'Référence prix': ref_prix,
                'Coefficient': coeff if i == 0 else '',
                'freq_totale': freq_totale_val if i == 0 else '',
                'nbr_equipe': nbr_equipe_val if i == 0 else '',
                'Source Cell': g_cell.coordinate if i == 0 else '',
                'Formula Type': formula_type if i == 0 else ''
            })

In [9]:
# Créer le DataFrame final
tunnel_mar_prix_coefficient = pd.DataFrame(all_rows, columns=[
    'Tunnel', 'Marché', 'Référence prix', 'Coefficient',
    'freq_totale', 'nbr_equipe', 'Source Cell', 'Formula Type'
])

# Remplissage vers l'avant (comme forward-fill) des colonnes nécessaires --> ffill()
tunnel_mar_prix_coefficient['Source Cell'] = tunnel_mar_prix_coefficient['Source Cell'].replace('', np.nan).ffill()

# Remplacer les chaînes vides par NaN
cols_to_fill = ['Coefficient', 'freq_totale', 'nbr_equipe']
for col in cols_to_fill:
    tunnel_mar_prix_coefficient[col] = tunnel_mar_prix_coefficient[col].replace('', np.nan)

# Remplir les valeurs manquantes par groupe
def fill_group(group):
    for col in cols_to_fill:
        group[col] = group[col].ffill().bfill()
    return group

tunnel_mar_prix_coefficient = (
    tunnel_mar_prix_coefficient
    .groupby(['Tunnel', 'Source Cell'], dropna=False)
    .apply(fill_group)
    .reset_index(drop=True)
)


<ipython-input-9-165483627>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tunnel_mar_prix_coefficient[col] = tunnel_mar_prix_coefficient[col].replace('', np.nan)
<ipython-input-9-165483627>:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(fill_group)


In [10]:
# Exporter le DataFrame nettoyé en CSV
tunnel_mar_prix_coefficient.to_csv('tunMarCoefFreqEq.csv', index=False)


In [25]:
tunnel_mar_prix_coefficient[:1].columns

Index(['Tunnel', 'Marché', 'Référence prix', 'Coefficient', 'freq_totale',
       'nbr_equipe', 'Source Cell', 'Formula Type'],
      dtype='object')

In [36]:
corCdPrix=prevTun[[ 'Marché',  'Référence\nprix',       'Prix\nactuel']].set_index(['Marché',  'Référence\nprix'])
corCdPrix.columns=['prixAct']

In [37]:
tunMarPr=tunnel_mar_prix_coefficient.join(corCdPrix,on=['Marché',  'Référence prix'])
tunMarPr[:1]

,Tunnel,Marché,Référence prix,Coefficient,freq_totale,nbr_equipe,Source Cell,Formula Type,prixAct
0,Ambroise Paré,ContReg,CRC101,1.0,1.0,0.0,G10,direct,648.6


In [39]:
tunMarPr['nbFrCfPr']=tunMarPr['Coefficient']*tunMarPr['freq_totale']*tunMarPr['nbr_equipe']*tunMarPr['prixAct']
tunMarPrA=tunMarPr.groupby(['Marché', 'Tunnel',tunMarPr['Référence prix']])['nbFrCfPr'].sum()

In [41]:
tunMarPrA[1:100:10]

Marché    Tunnel              Référence prix
AEV       Ambroise Paré       CRE102                0.00
          Bicêtre             AEV08              1357.40
          Chennevières        AEFB11             1417.44
          Guy Môquet          AEPE20             6033.78
          La Défense          AEPO10            30678.92
          Moulin              AEPE20             6033.78
          Nanterre échangeur  AEPO11            19251.36
          Orly                ECPS12                0.00
Automate  Ambroise Paré       AUPR11              309.50
          Belle-Rive          AUPR47              976.75
Name: nbFrCfPr, dtype: float64